In [1]:
iterat = 7

for i in range(1000000):
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn import metrics


    import os
    TITANIC_PATH = os.path.join("datasets", "titanic")

    import pandas as pd

    def load_titanic_data(filename, titanic_path=TITANIC_PATH):
        csv_path = os.path.join(titanic_path, filename)
        return pd.read_csv(csv_path)

    train_data = load_titanic_data("train.csv")
    test_data = load_titanic_data("test.csv")



    y_train = train_data['Survived'].values
    X_train = train_data.drop('Survived', axis=1)
    from sklearn.preprocessing import LabelEncoder

    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)

    from sklearn.base import BaseEstimator, TransformerMixin

    # A class to select numerical or categorical columns 
    # since Scikit-Learn doesn't handle DataFrames yet
    class CustomDataFrameSelector(BaseEstimator, TransformerMixin):
        repl_name = {
            'Rev': 0,
            'Sir': 0,
            'Don': 0,
            'Lady': 0,
            'Jonkheer': 0,
            'the Countess': 0,
            'Ms': 0,
            'Capt': 0,
            'Mme': 0,
            'Mlle': 0,
            'Mr': 1,
            'Mrs': 2,
            'Miss': 3,
            'Master': 4,
            'Dr': 4,
            'Col': 5,
            'Major': 5
        }

        repl_parch = {
            6: 0,
            4: 0,
            5: 1,
            0: 2,
            3: 3,
            1: 4,
            2: 5
        }

        repl_embark = {
            'S': 0,
            'Q': 1,
            'C': 2
        }

        repl_sib = {
            8: 0,
            5: 0,
            4: 1,
            3: 2,
            0: 3,
            2: 4,
            1: 4
        }

        repl_cab = {
            'T': 0,
            'D': 1,
            'E': 1,
            'B': 1,
            'U': 2,
            'F': 3,
            'C': 4,
            'A': 5,
            'G': 6
        }

        repl_sex = {
            'male': 1,
            'famale': 0
        }

        def __init__(self):
            pass
        def fit(self, X, y=None):
            return self
        def transform(self, X):
            X['Name'] = X['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
            titles = X['Name'].unique()

            X['Age'].fillna(-1, inplace=True)
            medians = dict()
            for title in titles:
                median = X.Age[(X["Age"] != -1) & (X['Name'] == title)].median()
                medians[title] = median

            for index, row in X.iterrows():
                if row['Age'] == -1:
                    X.loc[index, 'Age'] = medians[row['Name']]

            X['Fare'].fillna(-1, inplace=True)
            medians = dict()
            for pclass in X['Pclass'].unique():
                median = X.Fare[(X["Fare"] != -1) & (X['Pclass'] == pclass)].median()
                medians[pclass] = median

            for index, row in X.iterrows():
                if row['Fare'] == -1:
                    X.loc[index, 'Fare'] = medians[row['Pclass']]

            X['Name'] = X['Name'].apply(lambda x: self.repl_name.get(x))
            X['Parch'] = X['Parch'].apply(lambda x: self.repl_parch.get(x))
            X['Embarked'] = X['Embarked'].apply(lambda x: self.repl_embark.get(x))
            X['SibSp'] = X['SibSp'].apply(lambda x: self.repl_sib.get(x))
            X['Sex'] = X['Sex'].apply(lambda x: self.repl_sex.get(x))

            X['Cabin'].fillna('U', inplace=True)
            X['Cabin'] = X['Cabin'].apply(lambda x: x[0])
            X['Cabin'] = X['Cabin'].apply(lambda x: self.repl_cab.get(x))


            return X[['Age', 'Name', 'Fare', 'Pclass', 'Parch', 'Embarked', 'SibSp', 'Cabin', 'Sex']]

    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import OneHotEncoder

    from sklearn.model_selection import StratifiedKFold

    seed=34235237
    kfold = StratifiedKFold(n_splits=10, random_state=seed, shuffle=True)


    X_copy = X_train
    y_copy = y_train

    from sklearn.svm import SVC
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    from scipy.stats.distributions import uniform, randint

    from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
    from xgboost import XGBClassifier



    # classifier = XGBClassifier()
    # X_copy = X_train

    # for i in range(1000):
        

    X_train = X_copy
    y_train = y_copy

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)




    # param_grid = {
    #             'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
    # }

    # grid = GridSearchCV(pipe, param_grid, cv=kfold)

    # grid.fit(X_train, y_train)

    num_pipeline = Pipeline([
            ("select_numeric", CustomDataFrameSelector()),
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
            # ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
        ])


    classifier = RandomForestClassifier(bootstrap=False,
                        n_jobs=-1)

    pipe = Pipeline([
            ('preprocessing', num_pipeline), 
            ('classifier', classifier)])

    pipe.fit(X_train, y_train)

    score = pipe.score(X_test, y_test)
    # print(score)
     
    if score > 0.87082:
        submission = pd.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": pipe.predict(test_data)
        })

        submission.to_csv("submission" + iterat + ".csv", index=False)
        iterat+=1






    # num_pipeline.fit_transform(X_train)


FileNotFoundError: [Errno 2] No such file or directory: 'datasets\\titanic\\train.csv'